## Import

In [1]:
import pandas as pd
import xlrd
import numpy as np
import matplotlib.pyplot as plt

## File name and experiment set-up

In [2]:
infected_file = r'data\MOI 0.01 drug screen 2.xlsx'
uninfected_file = r'data\Uninfected nCoV drug screen 2.xlsx'
n_plates = 7
n_columns = 12
n_repeats = 3
n_concentrations = 8
start_concentration = 50

## Prepare array of concentrations

In [17]:
concentrations = []
concentrations.append(start_concentration)
for i in range(1, n_concentrations):
    concentrations.append(concentrations[i-1]/2)

## Read data from excel file into nice dictionary of dataframes

In [19]:
def get_dict_from_excel(filename, n_plates, n_columns, infected=True):
    
    if not infected:
        _n_plates = n_plates+1
    else:
        _n_plates = n_plates
    
    start_identifier = 'Plate:'
    end_identifier = '~End'
    
    book = xlrd.open_workbook(filename=filename)
    sheet = book.sheet_by_index(0)
    row_titles = sheet.col_values(0)
    
    start_indices = []
    end_indices = []
    
    for i in range(len(row_titles)):
        row_title = row_titles[i]
        if row_title==start_identifier:
            start_indices.append(i)
        elif row_title==end_identifier:
            end_indices.append(i)
    
    data = {}
    
    for plate in range(_n_plates):
        
        vals = []
        start = start_indices[plate]+2
        end = end_indices[plate]-1
        
        for col in range(2, 2+n_columns):
            cells = sheet.col_slice(col, start, end)
            for cell in cells:
                vals.append(cell.value)
        
        vals = np.reshape(vals, (n_columns, end-start)).T
        df = pd.DataFrame(data=vals)
        
        for i in range(n_concentrations):
            df.rename(index={i:concentrations[i]}, inplace=True)
           
        if not infected:
            _plate = plate
        else:
            _plate = plate+1
        
        data[f'Plate {_plate}'] = df
        
    return data

In [22]:
inf_data = get_dict_from_excel(infected_file, n_plates, n_columns)
uninf_data = get_dict_from_excel(uninfected_file, n_plates, n_columns, infected=False)

## Do triplicate averaging

In [26]:
def do_triplicate_averaging(data, n_columns, n_repeats, n_plates, infected=True):
    
    if not infected:
        _n_plates = n_plates+1
    else:
        _n_plates = n_plates
    
    data_averaged = {}
    
    n_samples = int(n_columns/n_repeats)
    
    for plate in range(_n_plates):
        df = pd.DataFrame()
        
        if not infected:
            _plate = plate
        else:
            _plate = plate+1
        
        for n in range(n_samples):
            i = n*n_repeats
            j = (n+1)*n_repeats-1
            
            av_col = data[f'Plate {_plate}'].loc[:,i:j].T.mean()
            df[n] = av_col
        
        data_averaged[f'Plate {_plate}'] = df
    
    return data_averaged

In [31]:
inf_data_averaged = do_triplicate_averaging(inf_data, n_columns, n_repeats, n_plates)
uninf_data_averaged = do_triplicate_averaging(uninf_data, n_columns, n_repeats, n_plates, infected=False)

## DMSO normalisations

In [48]:
def divide_by_dmso(data_averaged, n_plates, infected=True):
    
    if not infected:
        _n_plates = n_plates+1
    else:
        _n_plates = n_plates
    
    data_dmso_norm = {}

    for plate in range(_n_plates):
        
        if not infected:
            _plate = plate
        else:
            _plate = plate+1
        
        df = data_averaged[f'Plate {_plate}']
        data_dmso_norm[f'Plate {_plate}'] = df.iloc[:,:].div(df[3], axis=0)
        
    return data_dmso_norm

def subtract_by_dmso(data_averaged, n_plates, infected=True):
    
    if not infected:
        _n_plates = n_plates+1
    else:
        _n_plates = n_plates
    
    data_dmso_norm = {}

    for plate in range(_n_plates):
        
        if not infected:
            _plate = plate
        else:
            _plate = plate+1
        
        df = data_averaged[f'Plate {_plate}']
        data_dmso_norm[f'Plate {_plate}'] = df.iloc[:,:].subtract(df[3], axis=0)
        
    return data_dmso_norm

In [52]:
inf_data_dmso_div = divide_by_dmso(inf_data_averaged, n_plates)
uninf_data_dmso_div = divide_by_dmso(uninf_data_averaged, n_plates, infected=False)
inf_data_dmso_sub = subtract_by_dmso(inf_data_averaged, n_plates)
uninf_data_dmso_sub = subtract_by_dmso(uninf_data_averaged, n_plates, infected=False)

## Calculate % inhibition

In [53]:
inf_plate = inf_data_dmso_div['Plate 1']
uninf_plate = uninf_data_dmso_div['Plate 1']

inf_array = np.asarray(inf_plate).T
uninf_array = np.asarray(uninf_plate).T

inhib = np.divide(inf_array, uninf_array)
inhib = np.multiply(inhib, 100)

In [54]:
inf_plate

,0,1,2,3
50.000000,0.009497,1.894879,1.127979,1.0
25.000000,1.512653,1.409406,1.776586,1.0
12.500000,2.121985,1.515805,1.991296,1.0
6.250000,1.240623,1.108022,1.690663,1.0
3.125000,0.827656,0.895611,1.220886,1.0
1.562500,0.907916,0.966115,1.118834,1.0
0.781250,0.885305,0.940592,1.114223,1.0
0.390625,0.956920,1.008575,1.176975,1.0


In [55]:
uninf_plate

,0,1,2,3
50.000000,0.631379,0.955689,0.547280,1.0
25.000000,1.663929,1.170285,1.012071,1.0
12.500000,1.323861,1.131609,1.129387,1.0
6.250000,1.096840,1.105646,1.072778,1.0
3.125000,1.128027,1.100001,1.074990,1.0
1.562500,1.087247,1.039081,1.035038,1.0
0.781250,1.061276,1.091753,1.062606,1.0
0.390625,1.099908,1.090711,1.056209,1.0
